## 1. Постановка задачи



<center> <img src=https://raw.githubusercontent.com/hoittoken/Python/master/Py/Projects/project_5/logo%20v1.1.png align="left"> </center>

## 2. Знакомство с данными, базовый анализ и расширение данных

Начнём наше исследование со знакомства с предоставленными данными. А также подгрузим дополнительные источники данных и расширим наш исходный датасет. 


Заранее импортируем модули, которые нам понадобятся для решения задачи:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

from lightgbm import LGBMRegressor

## Напишем всякие полезные функции:

In [2]:
def add_datetime_features(data, feature='pickup_datetime'):
    
    data['pickup_date'] = data[feature].dt.date
    data['pickup_hour'] = data[feature].dt.hour
    data['pickup_day_of_week'] = data[feature].dt.dayofweek
    
    return data

In [3]:
def add_holiday_features(df, holiday_data):
    holiday_data['date'] = pd.to_datetime(holiday_data['date']).dt.date
    df = df.merge(holiday_data, how='left', left_on='pickup_date', right_on='date')
    df['pickup_holiday'] = df['holiday'].fillna(0)
    df['pickup_holiday'] = df['pickup_holiday'].apply(lambda x: 0 if x == 0 else 1)
    df = df.drop(['day', 'date', 'holiday'], axis=1)
    return df   

In [4]:
def add_osrm_features(data, data_2, on='id', how='left', columns = ['id','total_distance', 'total_travel_time', 'number_of_steps']):
    merged_data = data.merge(
        data_2[columns],
        on=on,
        how=how
    )
    return merged_data

In [5]:
def get_haversine_distance(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # радиус земли в километрах
    EARTH_RADIUS = 6371 
    # считаем кратчайшее расстояние h по формуле Хаверсина
    lat_delta = lat2 - lat1
    lng_delta = lng2 - lng1
    d = np.sin(lat_delta * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng_delta * 0.5) ** 2
    h = 2 * EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def get_angle_direction(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # считаем угол направления движения alpha по формуле угла пеленга
    lng_delta_rad = lng2 - lng1
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    alpha = np.degrees(np.arctan2(y, x))
    return alpha

def add_geographical_features(
    data, 
    lat1='pickup_latitude', 
    lng1='pickup_longitude',
    lat2='dropoff_latitude', 
    lng2='dropoff_longitude', 
    name_1='haversine_distance', name_2='direction'):
    
    data[name_1] = get_haversine_distance(data[lat1], data[lng1], data[lat2], data[lng2])
    
    data[name_2] = get_angle_direction(data[lat1], data[lng1], data[lat2], data[lng2])
    
    return data

In [6]:
def add_cluster_features(taxi_data):
    coords = np.hstack((taxi_data[['pickup_latitude', 'pickup_longitude']],
                    taxi_data[['dropoff_latitude', 'dropoff_longitude']]))
    # обучаем алгоритм кластеризации
    kmeans = cluster.KMeans(n_clusters=10, random_state=42)
    kmeans.fit(coords)
    pred = kmeans.predict(coords)
    pred = kmeans.labels_
    taxi_data['geo_cluster'] = pred
    return taxi_data


In [7]:
def add_weather_features(taxi_data, weather_data):
    
    weather_data['time'] = pd.to_datetime(weather_data['time'], format='%Y-%m-%d %H:%M:%S')
    taxi_data['date_&_hour'] = [(x, y) for x, y in zip(taxi_data['pickup_datetime'].dt.date, taxi_data['pickup_datetime'].dt.hour)]
    weather_data['date_&_hour'] = [(x, y) for x, y in zip(weather_data['time'].dt.date, weather_data['time'].dt.hour)]
    
    weather_data = weather_data.loc[:, ['date_&_hour', 'temperature', 'visibility', 'wind speed', 'precip', 'events']] 
    
    df = taxi_data.merge(
        weather_data,
        on = 'date_&_hour',
        how = 'left'
    )
    df.drop('date_&_hour', axis=1, inplace=True)
    return df

In [8]:
def fill_null_weather_data(taxi_data):
    cols_with_null = taxi_data.isnull().mean()
    cols_with_null = cols_with_null[cols_with_null > 0]
    for col in cols_with_null.index:
        if col == 'events':
            taxi_data[col] = taxi_data[col].fillna('None')
        else:
            taxi_data[col] = taxi_data[col].fillna(taxi_data.groupby('pickup_date')[col].transform('median'))
    return taxi_data

In [9]:
from sklearn.preprocessing import OneHotEncoder

scaler = preprocessing.MinMaxScaler()
one_hot_encoder = OneHotEncoder()

columns_to_change = ['pickup_day_of_week', 'geo_cluster', 'events']

best_features = ['pickup_hour', 'dropoff_latitude', 'pickup_longitude', 
 'haversine_distance', 'pickup_latitude', 'dropoff_longitude',
 'total_distance', 'total_travel_time', 'temperature', 'number_of_steps',
 'pickup_day_of_week_4', 'pickup_day_of_week_5', 'pickup_day_of_week_6',
 'vendor_id', 'passenger_count', 'pickup_day_of_week_3', 'geo_cluster_9',
 'pickup_day_of_week_1', 'pickup_day_of_week_2', 'geo_cluster_1',
 'geo_cluster_3', 'geo_cluster_5', 'geo_cluster_7', 'store_and_fwd_flag'
 ]

## Подготовим данные для обучения модели

In [10]:
osrm_data_train = pd.read_csv("C:\Личные документы\Учёба SF DS\data\osrm_data_train.zip")
train_data = pd.read_csv("C:\Личные документы\Учёба SF DS\data\Project5_train_data.zip")
holiday_data = pd.read_csv('holiday_data.csv', sep=";")
weather_data = pd.read_csv('weather_data.zip')

In [11]:
# Train set prepare

train_data['pickup_datetime']=pd.to_datetime(train_data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
train_data = add_datetime_features(train_data)
train_data = add_holiday_features(train_data, holiday_data)
train_data = add_osrm_features(train_data, osrm_data_train)
train_data = add_geographical_features(train_data)
train_data = add_cluster_features(train_data)
train_data = add_weather_features(train_data, weather_data)
train_data = fill_null_weather_data(train_data)

train_data['vendor_id'] = train_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)
train_data_onehot = one_hot_encoder.fit_transform(train_data[columns_to_change]).toarray()
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
train_data_onehot = pd.DataFrame(train_data_onehot, columns=column_names)

train_data = pd.concat(
    [train_data.reset_index(drop=True).drop(columns_to_change, axis=1), train_data_onehot], 
    axis=1
)
train_data = train_data.dropna(axis=1)
train_data['trip_duration_log'] = np.log(train_data['trip_duration']+1)

In [12]:
X = train_data[best_features]
scaler.fit(X)
X_test_scaled = scaler.transform(X)
y = train_data['trip_duration']
y_log = train_data['trip_duration_log']

In [13]:
X_train, X_valid, y_train_log, y_valid_log = model_selection.train_test_split(
    X, y_log, 
    test_size=0.33, 
    random_state=42
)

In [15]:
import xgboost as xgb
# Создание матриц наблюдений в формате DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train_log, feature_names=best_features)
dvalid = xgb.DMatrix(X_valid, label=y_valid_log, feature_names=best_features)


In [17]:
# Гиперпараметры модели
xgb_pars = {'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.9, 
            'max_depth': 6, 'subsample': 0.9, 'lambda': 1, 'nthread': -1, 
            'booster' : 'gbtree', 'eval_metric': 'rmse', 'objective': 'reg:squarederror'
           }
# Тренировочная и валидационная выборка
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
# Обучаем модель XGBoost
model = xgb.train(
    params=xgb_pars, #гиперпараметры модели
    dtrain=dtrain, #обучающая выборка
    num_boost_round=400, #количество моделей в ансамбле
    evals=watchlist, #выборки, на которых считается матрица
    early_stopping_rounds=20, #раняя остановка
    maximize=False, #смена поиска максимума на минимум
    verbose_eval=10 #шаг, через который происходит отображение метрик
)

[0]	train-rmse:5.42171	valid-rmse:5.42130
[10]	train-rmse:1.93626	valid-rmse:1.93698
[20]	train-rmse:0.78730	valid-rmse:0.78843
[30]	train-rmse:0.48194	valid-rmse:0.48319
[40]	train-rmse:0.42488	valid-rmse:0.42616
[50]	train-rmse:0.41277	valid-rmse:0.41424
[60]	train-rmse:0.40816	valid-rmse:0.41017
[70]	train-rmse:0.40449	valid-rmse:0.40684
[80]	train-rmse:0.40270	valid-rmse:0.40546
[90]	train-rmse:0.40077	valid-rmse:0.40378
[100]	train-rmse:0.39917	valid-rmse:0.40249
[110]	train-rmse:0.39793	valid-rmse:0.40160
[120]	train-rmse:0.39636	valid-rmse:0.40039
[130]	train-rmse:0.39500	valid-rmse:0.39932
[140]	train-rmse:0.39404	valid-rmse:0.39863
[150]	train-rmse:0.39324	valid-rmse:0.39815
[160]	train-rmse:0.39271	valid-rmse:0.39788
[170]	train-rmse:0.39190	valid-rmse:0.39747
[180]	train-rmse:0.39112	valid-rmse:0.39702
[190]	train-rmse:0.39039	valid-rmse:0.39665
[200]	train-rmse:0.38963	valid-rmse:0.39627
[210]	train-rmse:0.38891	valid-rmse:0.39585
[220]	train-rmse:0.38822	valid-rmse:0.39552

In [146]:
l_model = LGBMRegressor(
    objective='regression',
    metric='rmse',
    learning_rate=0.1,
    max_depth=25,
    num_leaves=1000, 
    feature_fraction=0.9,
    bagging_fraction=0.5,
    max_bin=1000 ,
    num_threads=-1)
l_model.fit(X_train, y_train_log)


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1


LGBMRegressor(bagging_fraction=0.5, feature_fraction=0.9, learning_rate=0.08,
              max_bin=1000, max_depth=30, metric='rmse', num_leaves=1000,
              num_threads=-1, objective='regression')

In [ ]:
params = {
    'objective': 'regression',
    'metric' : 'rmse',
    'learning_rate': 0.1,
    'max_depth': 25,
    'num_leaves': 1000, 
    'feature_fraction': 0.9,
    'bagging_fraction': 0.5,
    'max_bin': 1000 ,
    'num_threads' : -1}

In [157]:
params = {}

In [170]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, y_train_log)
eval_data = lgb.Dataset(X_valid, y_valid_log)

l_model = lgb.train(params, train_data, num_boost_round=1500, valid_sets=(train_data, eval_data),
                early_stopping_rounds=110, verbose_eval=100)

c:\Users\AubakirovMA\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\AubakirovMA\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024764 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1993
[LightGBM] [Info] Number of data points in the train set: 729322, number of used features: 24
[LightGBM] [Info] Start training from score 6.466908
Training until validation scores don't improve for 110 rounds
[100]	training's l2: 0.161415	valid_1's l2: 0.163781
[200]	training's l2: 0.153552	valid_1's l2: 0.158945
[300]	training's l2: 0.148455	valid_1's l2: 0.156262
[400]	training's l2: 0.144477	valid_1's l2: 0.154669
[500]	training's l2: 0.141361	valid_1's l2: 0.153496
[600]	training's l2: 0.138437	valid_1's l2: 0.152485
[700]	training's l2: 0.136214	valid_1's l2: 0.151941
[800]	training's l2: 0.134079	valid_1's l2: 0.151317
[900]	training's l2: 0.132185	valid_1's l2: 0.150867
[1000]	training's l2: 0.130419	valid_1's l2: 0.1505

In [165]:
y_train_lgb = l_model.predict(X_train)
y_valid_lgb = l_model.predict(X_valid)

In [140]:
y_train_xgb = model.inplace_predict(X_train)
y_valid_xgb = model.inplace_predict(X_valid)

y_train_lgb = l_model.predict(X_train)
y_valid_lgb = l_model.predict(X_valid)


In [167]:

y_train_pred = (y_train_xgb + y_train_lgb) / 2
y_valid_pred = (y_valid_xgb + y_valid_lgb) / 2

In [166]:
from sklearn.metrics import median_absolute_error

def rmsle(y_real, y_pred):
    mse = np.sum(np.square(np.subtract(y_real, y_pred))) / len(y_real)
    return np.sqrt(mse)  

y_valid_real = np.exp(y_valid_log)-1
y_valid_pred_real = np.exp(y_valid_lgb) - 1

MeAE = median_absolute_error(y_valid_real, y_valid_pred_real) / 60

print(f"RMSLE on train by XGB: {rmsle(y_train_log, y_train_xgb)}")
print(f"RMSLE on valid by XGB: {rmsle(y_valid_log, y_valid_xgb)}")
print()
print(f"RMSLE on train by LGB: {rmsle(y_train_log, y_train_lgb)}")
print(f"RMSLE on valid by LGB: {rmsle(y_valid_log, y_valid_lgb)}")
print()
print(f"MeAE on valid: {round(MeAE, 2)}")

RMSLE on train by XGB: 0.3780398208476835
RMSLE on valid by XGB: 0.3937419406785572

RMSLE on train by LGB: 0.35082873860493413
RMSLE on valid by LGB: 0.38651539370263915

MeAE on valid: 1.75


In [169]:
from sklearn.metrics import median_absolute_error

y_valid_real = np.exp(y_valid_log)-1
y_valid_pred_real = np.exp(y_valid_pred) - 1

def rmsle(y_real, y_pred):
    mse = np.sum(np.square(np.subtract(y_real, y_pred))) / len(y_real)
    return np.sqrt(mse)    

MeAE = median_absolute_error(y_valid_real, y_valid_pred_real) / 60

print(f"RMSLE on train: {rmsle(y_train_log, y_train_pred)}")
print(f"RMSLE on valid: {rmsle(y_valid_log, y_valid_pred)}")
print(f"MeAE on valid: {round(MeAE, 2)}")

RMSLE on train: 0.3617577161751215
RMSLE on valid: 0.3872130780646488
MeAE on valid: 1.78


# Тестовые данные для kaggle

In [104]:
osrm_data_test = pd.read_csv("C:\Личные документы\Учёба SF DS\data\Project5_osrm_data_test.zip")
test_data = pd.read_csv("C:\Личные документы\Учёба SF DS\data\Project5_test_data.zip")
holiday_data = pd.read_csv('holiday_data.csv', sep=";")
weather_data = pd.read_csv('weather_data.zip')
test_id = test_data['id']

In [105]:
test_data['pickup_datetime']=pd.to_datetime(test_data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
test_data = add_datetime_features(test_data)
test_data = add_holiday_features(test_data, holiday_data)
test_data = add_osrm_features(test_data, osrm_data_test)
test_data = add_geographical_features(test_data)
test_data = add_cluster_features(test_data)
test_data = add_weather_features(test_data, weather_data)
test_data = fill_null_weather_data(test_data)

test_data['vendor_id'] = test_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)
test_data['store_and_fwd_flag'] = test_data['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)
test_data_onehot = one_hot_encoder.fit_transform(test_data[columns_to_change]).toarray()
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
test_data_onehot = pd.DataFrame(test_data_onehot, columns=column_names)

test_data = pd.concat(
    [test_data.reset_index(drop=True).drop(columns_to_change, axis=1), test_data_onehot], 
    axis=1
)
X_test = test_data[best_features]
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)
print('Shape of data: {}'.format(X_test.shape))

Shape of data: (625134, 24)


In [107]:
y_test_xgb = model.inplace_predict(X_test)

y_test_lgb = booster.predict(X_test)

y_test_pred = (y_test_xgb + y_test_lgb) / 2

In [109]:
y_test_predict = np.exp(y_test_pred) - 1

In [110]:
submission = pd.DataFrame({'id': test_id, 'trip_duration': y_test_predict})
submission.to_csv('submission_gb.csv', index=False)

# Private Score: 0.40158   
# Public Score: 0.40414